In [1]:
# !pip install pip install jupyter
# !pip install ipywidgets widgetsnbextension pandas-profiling
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix

# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [2]:
# !pip install --quiet  datasets # to access squad dataset
# !pip install --quiet pyarrow   # to deal with parquet files for saving dataset if required
# !pip install --quiet  tqdm     # for progress bars
# !pip install --quiet transformers # for t5 model
# !pip install --quiet tokenizers  # tokenizers from HuggingFace
# !pip install --quiet sentencepiece # subword tokenizer used by T5
# !pip install --quiet pytorch-lightning # pytorch wrapper 
# !pip install --quiet torchtext # text utilities
# !pip install pandas datasets pyarrow tqdm transformers tokenizers sentencepiece torchtext

# # watch -n 1 free -h
# # watch -n 1 nvidia-smi

# # torch.cuda.empty_cache() 

# Fetching Datasets

In [3]:
import os
import random

import numpy as np
import torch

import pandas as pd

from datasets import DatasetDict, Dataset, load_from_disk
from tokenizers import AddedToken
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, EarlyStoppingCallback
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers.optimization import Adafactor, AdafactorSchedule

# from ../evaluation_metrics import Metrics


seed = 42
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# pd.options.display.max_rows , pd.options.display.max_columns  = 100,100  

device = 'cuda' if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
import ipywidgets as widget
widget.IntSlider()

IntSlider(value=0)

In [5]:
from huggingface_hub import notebook_login

notebook_login()

# Check the data 


In [7]:
# df = pd.read_parquet('train_tdm_f1_v1.parquet')#.iloc[:2000,:]
# df = pd.read_parquet('train_squad.parquet')

# mode = "tdm"
mode = "tdms"
train_path = f'../data/train_{mode}_f1_v2_short.parquet' 
validation_path = f'../data/dev_{mode}_f1_v2_short.parquet'

df = pd.read_parquet(f'{train_path}')

df

,template_question,answer
0,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
1,Read this and answer the question. If the ques...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
2,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
3,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
4,Value Prediction Network This paper proposes a...,[{'leaderboard': {'Dataset': 'Atari 2600 Seaqu...
...,...,...
82675,LogMaster: Mining Event Correlations in Logs o...,"[{'leaderboard': {'Dataset': '', 'Metric': '',..."
82676,LogMaster: Mining Event Correlations in Logs o...,"[{'leaderboard': {'Dataset': '', 'Metric': '',..."
82677,Read this article and answer this question Log...,"[{'leaderboard': {'Dataset': '', 'Metric': '',..."
82678,LogMaster: Mining Event Correlations in Logs o...,"[{'leaderboard': {'Dataset': '', 'Metric': '',..."


In [8]:
str(df.at[5, 'answer'])

"[{'leaderboard': {'Dataset': 'Atari 2600 Seaquest', 'Metric': 'Score', 'Score': '5628', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Amidar', 'Metric': 'Score', 'Score': '641', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Krull', 'Metric': 'Score', 'Score': '15930', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Alien', 'Metric': 'Score', 'Score': '1429', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Enduro', 'Metric': 'Score', 'Score': '382', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Ms. Pacman', 'Metric': 'Score', 'Score': '2689', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Crazy Climber', 'Metric': 'Score', 'Score': '54119', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Q*Bert', 'Metric': 'Score', 'Score': '14517', 'Task': 'Atari Games'}}\n {'leaderboard': {'Dataset': 'Atari 2600 Frostbite', 'Metric': 'Score', 'Score': '3811', 'Task': 'A

# Creating a Pytorch DataSet for T5 Training and Validation

In [9]:
# Number of CPU cores
workers = os.cpu_count()
workers

48

In [10]:
model_name = ["google/flan-t5", "google/long-t5"]
size = ["-base", "-large", "-xl"]
model_attention = ["","-local", "-tglobal"]


bs = 32
epochs = 5

# gpus = 1
# workers = os.cpu_count()

gpus = -1
workers = os.cpu_count()
# workers = 0

model_idx = 0
size_idx = 1
model_attent_idx = 0


# # # # model_max_length = None
# # model_max_length = 6000
# # max_len_inp = 5750
# # model_max_len_out = 250

# model_max_length = 512
# max_len_inp = 512
# # max_len_inp = 500
# # max_len_inp = 512
# # model_max_len_out = 130
# model_max_len_out = 512

# model_id = "google/flan-t5-large"
model_id = model_name[model_idx]+size[size_idx]
max_source_length = 512
max_target_length = 512
label_pad_token_id = -100
model_max_length = 512

dataset_path = "../data/LLLM_TDMS_ALL_TEMPLATE/"
metric_name = "partial_f1s_overall"

# fine_tuned_model_repository = f"{dataset_path}"
# fine_tuned_model_repository = f"{path_prefix}/models/text_based/{model_is_from_train_set}"
# tokenizer_repository = f"{path_prefix}/models/text_based/{model_is_from_train_set}/tokenizer"

tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.add_tokens(AddedToken("\n", normalized=False))
tokenizer.add_tokens(AddedToken("{", normalized=False))
tokenizer.add_tokens(AddedToken("}", normalized=False))

model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


# t5_tokenizer = AutoTokenizer.from_pretrained(
#     f"{model_name[model_idx]}{model_attention[model_idx]}{size[size_idx]}", model_max_length=model_max_length)
# t5_model = T5ForConditionalGeneration.from_pretrained(f"{model_name[model_idx]}{model_attention[model_idx]}{size[size_idx]}")

# t5_tokenizer.add_tokens(AddedToken("{", normalized=False))
# t5_tokenizer.add_tokens(AddedToken("}", normalized=False))

# t5_model = LongT5Model.from_pretrained(f"{model[model_idx]}{model_attention[model_idx]}{size[size_idx]}")
# t5_model = LongT5ForConditionalGeneration.from_pretrained(f"{model[model_idx]}{model_attention[model_idx]}{size[size_idx]}")

print(f"Max token lenght: {tokenizer.model_max_length}")

Max token lenght: 512


In [11]:
print(f"Max token lenght: {tokenizer.model_max_length}")
print(f"Batch size: {bs}")
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")

Max token lenght: 512
Batch size: 32
Number of GPUs available: 4


In [12]:
model_save_path = f'../model_ckpt/{model_name[model_idx].replace("/","_").replace("-","_")}{model_attention[model_attent_idx].replace("-","_")}{size[size_idx].replace("-","_")}_{bs}_{epochs}_{model_max_length}_{max_source_length}_{max_target_length}_gpu_{num_gpus}'
tokenizer_repository = f'../model_ckpt/{model_name[model_idx].replace("/","_").replace("-","_")}{model_attention[model_attent_idx].replace("-","_")}{size[size_idx].replace("-","_")}_tokenizer_{bs}_{epochs}_{model_max_length}_{max_source_length}_{max_target_length}_gpu_{num_gpus}'

In [13]:
def tokenize_function(sample):
    # tokenize inputs
    model_inputs = tokenizer(sample["prompt"], max_length=max_source_length, padding="max_length", truncation=True,
                             return_tensors="pt")

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["answer"], max_length=max_target_length, padding="max_length",
                       truncation=True, return_tensors="pt")

    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]  # type: ignore
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [14]:
# train_dataset = pd.read_parquet(f'{train_path}')
# eval_dataset = pd.read_parquet(f'{validation_path}')


dataset = DatasetDict.load_from_disk(f"{dataset_path}/fold1")
dataset = dataset.shuffle(seed=seed)
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

In [15]:
def compute_metrics(eval_preds):

    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)  # type: ignore
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)  # type: ignore
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = Metrics.evaluate_property_wise_text_based(label_list=decoded_labels, prediction_list=decoded_preds)
    result.update(Metrics.evaluate_rouge(label_list=decoded_labels, prediction_list=decoded_preds))
    return result

In [16]:
train_tokenized_dataset = train_dataset.map(tokenize_function, batched=True,
                                            # remove_columns=dataset_columns_to_remove
                                            )
eval_tokenized_dataset = eval_dataset.map(tokenize_function, batched=True,
                                        #   remove_columns=dataset_columns_to_remove
                                          )
print(f"Keys of tokenized dataset: {list(train_tokenized_dataset.features)}")

optimizer = Adafactor(model.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)
lr_scheduler = AdafactorSchedule(optimizer)
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10, early_stopping_threshold=0.001)

Keys of tokenized dataset: ['prompt', 'answer', '__index_level_0__', 'input_ids', 'attention_mask', 'labels']


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=model_save_path,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    # per_device_train_batch_size=2,
    # per_device_eval_batch_size=4,
    predict_with_generate=True,
    num_train_epochs=30,
    generation_max_length=model_max_length,
    gradient_accumulation_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    greater_is_better=True,
    logging_dir=f"{model_save_path}/logs",
    logging_strategy="steps",
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    # report_to="tensorboard",
    push_to_hub=False,
    seed=seed
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized_dataset,
    eval_dataset=eval_tokenized_dataset,
    # compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler),
    callbacks=([early_stopping_callback])

)

trainer.train()
tokenizer.save_pretrained(tokenizer_repository)
best_ckpt_path = trainer.state.best_model_checkpoint
print(f"best epoch: {best_ckpt_path}")

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/root/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


In [ ]:
# !pip install transformers[torch]

In [ ]:
#imports
import pandas as pd
import torch, os
from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy
import numpy as np

from collections import defaultdict
import ipdb
import random

import pytorch_lightning as pl
from torch.optim import AdamW
import argparse
from transformers import (
    get_linear_schedule_with_warmup
  )

from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

from tokenizers import AddedToken

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup,
    AutoTokenizer, 
    LongT5Model,
    LongT5ForConditionalGeneration
) 




In [ ]:
# df["Lenght answer"] = df.answer.apply(lambda x: len(str(x)))
# df["Lenght answer"] = df.answer.apply(lambda x: len(str(x).split()))
# df["Lenght question"] = df.template_question.apply(lambda x: len(x.split()))

# # df["Lenght answer"] = df.answer.apply(lambda x: len(x))
# # df["Lenght question"] = df.template_question.apply(lambda x: len(x))

# df["Lenght answer"] = df.answer.apply(lambda x: len(t5_tokenizer.batch_encode_plus(
#                 [str(x)], 
#                 truncation = False,
#                 return_tensors="pt"
#             )['input_ids'][0]))
# df["Lenght question"] = df.template_question.apply(lambda x: len(t5_tokenizer.batch_encode_plus(
#                 [str(x)], 
#                 truncation = False,
#                 return_tensors="pt"
#             )['input_ids'][0]))

# df.describe()

In [ ]:
class QuestionGenerationDataset(Dataset):
    def __init__(self, tokenizer, filepath, max_len_inp=512,max_len_out=96):
        self.path = filepath

        self.passage_column = "context"
        self.answer = "answer"
        self.question = "question"
        self.template_question = "template_question"

        # self.data = pd.read_csv(self.path)
        self.data = pd.read_parquet(self.path)

        self.max_len_input = max_len_inp
        self.max_len_output = max_len_out
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  #squeeze to get rid of the batch dimension
        target_mask = self.targets[index]["attention_mask"].squeeze()  # convert [batch,dim] to [dim] 

        labels = copy.deepcopy(target_ids)
        labels [labels==0] = -100

        return {
            "source_ids": source_ids, "source_mask": src_mask, 
            "target_ids": target_ids, "target_mask": target_mask,"labels":labels}

    def _build(self):
        for rownum,val in tqdm(self.data.iterrows(), total=len(self.data)): # Iterating over the dataframe
#             passage,answer,target = val[self.passage_column],val[self.answer],val[self.question]
            template_question, answer = val[self.template_question], val[self.answer]

#             input_ = f"context: {passage}  answer: {answer}" # T5 Input format for question answering tasks 
#             target = f"question: {str(target)}" # Output format we require
            
#             input_ = f"context: {passage} question: {str(target)}" # T5 Input format for question answering tasks 
#             target = f"answer: {answer}" # Output format we require
            
#             input_ = f"{str(template_question)} </s>" # T5 Input format for question answering tasks 
#             target = f"{answer} </s>" # Output format we require
            input_ = f"{str(template_question)}" # T5 Input format for question answering tasks 
            target = f"{answer}" # Output format we require
            
            # TODO: Not sure if this is needed as the tokenizer can truncate the output. 
            encoded = t5_tokenizer.batch_encode_plus(
                [input_], 
                truncation = False,
                return_tensors="pt"
            )
            
            if len(encoded['input_ids'][0]) > self.max_len_output:
                continue 
                
#             question_plus = f"answer_me: {str(question)}"
#             question_plus += f" context: {str(context)} </s>"

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], 
                max_length = self.max_len_input,
                padding = 'max_length',
                truncation = True,
                return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], 
                max_length = self.max_len_output,
#                 padding = 'longest',
                padding = 'max_length',
                truncation = True,
#                 truncation = False,
                return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [ ]:
# # train_path = 'train_squad.parquet' # change this accordingly
# # validation_path = 'validation_squad.parquet'

# # train_path = 'train_tdm_f1_v1.parquet' # change this accordingly
# # train_path = 'dev_tdm_f1_v1.parquet' # change this accordingly
# # validation_path = 'dev_tdm_f1_v1.parquet'

# # train_path = 'train_tdm_f1_v1_short.parquet'
# train_path = 'data/train_tdm_f1_v2_short.parquet'
# validation_path = 'data/dev_tdm_f1_v2_short.parquet'


# train_dataset = QuestionGenerationDataset(t5_tokenizer, train_path, 
#                                           max_len_inp=max_len_inp, max_len_out=model_max_len_out)
# # validation_dataset = QuestionGenerationDataset(t5_tokenizer, validation_path, 
# #                                                max_len_inp=max_len_inp, max_len_out=model_max_len_out)

# validation_dataset = train_dataset


train_dataset = QuestionGenerationDataset(t5_tokenizer, train_path, 
                                          max_len_inp = max_len_inp, 
                                          max_len_out = model_max_len_out)
validation_dataset = QuestionGenerationDataset(t5_tokenizer, validation_path, 
                                               max_len_inp = max_len_inp, 
                                               max_len_out = model_max_len_out)


In [ ]:
# Data Sample

train_sample = train_dataset[100] # thanks to __getitem__
decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

print(decoded_train_input)
print(decoded_train_output)

In [ ]:
# Data Sample

train_sample = train_dataset[100] # thanks to __getitem__
decoded_train_input = t5_tokenizer.decode(train_sample['source_ids'])
decoded_train_output = t5_tokenizer.decode(train_sample['target_ids'])

print(decoded_train_input)
print(decoded_train_output)

In [ ]:
print(f"Length training {len(train_dataset)}")
print(f"Length validation {len(validation_dataset)}")

In [ ]:
len("[{ 'leaderboard': { 'Dataset': 'Human3.6M', 'Metric': 'Average MPJPE (mm)', 'Task</s>")

# Fine Tuning T5

In [ ]:
import pytorch_lightning as pl
from torch.optim import AdamW
import argparse
from transformers import (
    get_linear_schedule_with_warmup
  )

# from transformers import get_linear_schedule_with_warmup, \
# T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup

class T5Tuner(pl.LightningModule):
    # 3e-4
    def __init__(self, t5model, t5tokenizer, batchsize=4):
#     def __init__(self, t5model, t5tokenizer, 
#                  lr=5e-5, num_train_epochs=15, warmup_steps=1000, batchsize=4):
#     super(T5Tuner, self).__init__()    
        super().__init__()
        self.model = t5model
        self.tokenizer = t5tokenizer
        self.batch_size = batchsize
#         self.save_hyperparameters()

    def forward(self, input_ids, attention_mask=None, 
                decoder_attention_mask=None, 
                lm_labels=None):
      
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
        )
        
        return outputs

    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log('train_loss',loss)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            lm_labels=batch['labels']
        )

        loss = outputs[0]
        self.log("val_loss",loss)
        return loss

    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.batch_size,
                          num_workers=workers)

    def val_dataloader(self):
        return DataLoader(validation_dataset, 
                          batch_size=self.batch_size,
                          num_workers=workers)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer
        
#     def configure_optimizers(self):
#         # create optimizer
#         optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
#         # create learning rate scheduler
#         num_train_optimization_steps = self.hparams.num_train_epochs * len(\
# self.train_dataloader())
#         lr_scheduler = {
# 'scheduler': get_linear_schedule_with_warmup(optimizer,
#                         num_warmup_steps=self.hparams.warmup_steps,
#                         num_training_steps=num_train_optimization_steps),
#                         'name': 'learning_rate',
#                         'interval':'step',
#                         'frequency': 1}
        
#         return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
    


In [ ]:
# device

In [ ]:
gpus

In [ ]:
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=5,
   strict=False,
   verbose=False,
   mode='min'
)

bs = 1


# # for early stopping, 
# # see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping\
# # .html?highlight=early%20stopping
# early_stop_callback = EarlyStopping(
#     monitor='validation_loss',
#     patience=3,
#     strict=False,
#     verbose=False,
#     mode='min'
# )

# lr_monitor = LearningRateMonitor(logging_interval='step')

model = T5Tuner(t5_model, t5_tokenizer, batchsize=bs)

trainer = pl.Trainer(max_epochs = epochs,
                     accelerator="auto", 
                     devices="auto",
                     # gpus=gpus,
                     strategy='dp',
                     # accelerator='cuda', 
                     # devices=gpus,
                     callbacks=[
                         early_stop_callback, 
#                          lr_monitor
                     ]
                    )

trainer.fit(model)

In [ ]:
# torch.cuda.empty_cache() 

In [ ]:
# # saving the model
# !mkdir "t5_tokenizer"
# !mkdir "t5_trained_model"
# model.model.save_pretrained('t5_trained_model_QA')
# model.model.save_pretrained('t5_trained_model_QA__')

# t5_tokenizer.save_pretrained('t5_tokenizer_QA')

In [ ]:
# model[model_idx].replace("/","_").replace("-","_")

In [ ]:
# model.model.save_pretrained(\
# f'../model_ckpt/{model[model_idx].replace("/","_").replace("-","_")}{model_attention[model_idx].replace("-","_")}{size[size_idx].replace("-","_")}_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}_gpu_{num_gpus}')
# f'../model_ckpt/{model_name[model_idx].replace("/","_").replace("-","_")}{model_attention[model_idx].replace("-","_")}{size[size_idx].replace("-","_")}_tokenizer_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}_gpu_{num_gpus}'

# Inference / Predictions

In [ ]:
!pwd

In [ ]:
# model.model.save_pretrained(f'model_ckpt/{mode}_{model_name[model_idx].replace("/","_").replace("-","_")}{model_attention[model_idx].replace("-","_")}{size[size_idx].replace("-","_")}_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}_gpu_{num_gpus}')
# # t5_tokenizer.save_pretrained(f'{model[model_idx]}{model_attention[model_idx]}{size[size_idx]}_tokenizer_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}')
# t5_tokenizer.save_pretrained(f'model_ckpt/{mode}_{model_name[model_idx].replace("/","_").replace("-","_")}{model_attention[model_idx].replace("-","_")}{size[size_idx].replace("-","_")}_tokenizer_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}_gpu_{num_gpus}')


In [ ]:
# trained_model_path = f'Longt5_trained_tdms_model_QA_bs_{bs}_epochs_{epochs}_model_max_length_{model_max_length}_max_len_inp_{max_len_inp}_model_max_len_out_{model_max_len_out}'
# trained_tokenizer = f'Longt5_tokenizer_tdms_QA_bs_{bs}_epochs_{epochs}_model_max_length_{model_max_length}_max_len_inp_{max_len_inp}_model_max_len_out_{model_max_len_out}'

# trained_model_path = f'./Longt5_trained_tdms_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}'
# trained_tokenizer = f'./Longt5_tokenizer_tdms_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}'

trained_model_path = f'../model_ckpt/{mode}_{model_name[model_idx].replace("/","_").replace("-","_")}{model_attention[model_idx].replace("-","_")}{size[size_idx].replace("-","_")}_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}_gpu_{num_gpus}'
trained_tokenizer = f'../model_ckpt/{mode}_{model_name[model_idx].replace("/","_").replace("-","_")}{model_attention[model_idx].replace("-","_")}{size[size_idx].replace("-","_")}_tokenizer_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}_gpu_{num_gpus}'

# trained_model_path = f'../model_ckpt/{model[model_idx].replace("/","_").replace("-","_")}{model_attention[model_idx].replace("-","_")}{size[size_idx].replace("-","_")}_{bs}_{epochs}_{model_max_length}_{max_len_inp}_{model_max_len_out}_gpu_{num_gpus}'
# trained_tokenizer = f'./Longt5_tokenizer_tdms_6_5_512_400_112'

# device  = 'cuda' if torch.cuda.is_available() else "cpu"
device = "cpu"

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)

In [ ]:
# context ="President Donald Trump said and predicted that some states would reopen this month."
# # answer = "Donald Trump"
# question = "Who is the pre"
# text = "context: "+context + " " + "answer: " + answer
# print(text)

# Data Sample Validation 

# dev_sample = validation_dataset[10] # thanks to __getitem__
dev_sample = train_dataset[100] # thanks to __getitem__


decoded_dev_input = t5_tokenizer.decode(dev_sample['source_ids'])
decoded_dev_output = t5_tokenizer.decode(dev_sample['target_ids'])

print(decoded_dev_input)
print(decoded_dev_output)

In [ ]:
# context ="Since its topping out in 2013, One World Trade Center in New York City has been the tallest skyscraper in the United States."
# # answer = "World Trade Center"
# # question = "When did the One World Trade Center became the tallest skyscraper"
# question = "Where is the world trade center ?"
# text = "context: "+context + " " + "answer: " + question
# # text = "context: "+context + " " + "answer: " + answer

# print(text)

In [ ]:
encoding = tokenizer.encode_plus(decoded_dev_input,max_length =512,padding='max_length', 
                                 truncation = True,
                                 return_tensors="pt")#.to(device)
print (encoding.keys())
# input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
input_ids,attention_mask  = encoding["input_ids"], encoding["attention_mask"]

In [ ]:
model.eval()
beam_outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=model_max_len_out, # How long the generated questions should be
    early_stopping=True,
    num_beams=10,
    num_return_sequences=1
)

for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(sent)